### Clear GPU

In [ ]:
from keras import backend as K

K.clear_session()

In [ ]:
import tensorflow as tf
from numba import cuda
import gc

def clear_memory():
    # Clear VRAM
    tf.keras.backend.clear_session()
    cuda.select_device(0)
    cuda.close()
    
    # Clear RAM
    gc.collect()

#This should clear the VRAM and RAM
clear_memory()

### Check GPU Existence and Status




In [ ]:
import tensorflow as tf

# List all GPUs TensorFlow detects
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("TensorFlow detected the following GPU(s):")
    for gpu in gpus:
        details = tf.config.experimental.get_device_details(gpu)
        print(f"Name: {details['device_name']}")

In [ ]:
#This is to check GPU-Status and Usage (works only for NVIDIA GPUs)
!nvidia-smi

physical_devices = tf.config.list_physical_devices('GPU')
for gpu in physical_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

### Check Tensorflow and Keras Version


In [ ]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

### Import all the libraries

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
import keras as tfk
from keras import layers as tfkl
from keras import regularizers

# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Print TensorFlow version
print(tf.__version__)

# Import other libraries
import requests
from io import BytesIO
import cv2
from PIL import Image
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

#Number of Classes in the Dataset
num_classes = 8

### Create a function to Load Data and load the datasets needed


In [ ]:
def load_data(path):
    # Load dataset with TensorFlow Datasets, obtaining dataset info

    data = np.load(path)
    train_dataset = data['images']
    test_dataset= data['labels']

    train_dataset = train_dataset[:11959]
    test_dataset = test_dataset[:11959]

    return (train_dataset,test_dataset)

In [ ]:
# Execute function and load data
(X_test, y_test) = load_data("training_set.npz")

print("Test set shape (images):", X_test.shape)
print("Test set shape (labels):", y_test.shape)

In [ ]:
# Execute function and load data
(X_test_aug, y_test_aug) = load_data("augmented_set.npz")

print("Test set shape (images):", X_test_aug.shape)
print("Test set shape (labels):", y_test_aug.shape)

In [ ]:
# Execute function and load data
(X_test_aug2, y_test_aug2) = load_data("augmented_set2.npz")

print("Test set shape (images):", X_test_aug2.shape)
print("Test set shape (labels):", y_test_aug2.shape)

In [ ]:
# Execute function and load data
(X_test_aug3, y_test_aug3) = load_data("augmented_set3.npz")

print("Test set shape (images):", X_test_aug3.shape)
print("Test set shape (labels):", y_test_aug3.shape)

In [ ]:
# Execute function and load data
(X_test_aug4, y_test_aug4) = load_data("augmented_set4.npz")

print("Test set shape (images):", X_test_aug4.shape)
print("Test set shape (labels):", y_test_aug4.shape)

### Different Combination of Datasets

In [ ]:
# Create different type of concatenations of the datasets
X_test_concat = np.concatenate((X_test, X_test_aug), axis=0)
y_test_concat = np.concatenate((y_test, y_test_aug), axis=0)

In [ ]:
# Concatenate datasets 
X_test_concat2 = np.concatenate((X_test_concat, X_test_aug2), axis=0)
y_test_concat2 = np.concatenate((y_test_concat, y_test_aug2), axis=0)

In [ ]:
# Concatenate datasets
X_test_concat3 = np.concatenate((X_test_concat, X_test_aug3), axis=0)
y_test_concat3 = np.concatenate((y_test_concat, y_test_aug3), axis=0)

In [ ]:
# Concatenate datasets -> All the datasets were made by me 
X_test_concat_d = np.concatenate((X_test_aug, X_test_aug3), axis=0)
y_test_concat_d = np.concatenate((y_test_aug, y_test_aug3), axis=0)
X_test_concat_d = np.concatenate((X_test_concat_d, X_test_aug4), axis=0)
y_test_concat_d = np.concatenate((y_test_concat_d, y_test_aug4), axis=0)

### One-hot Encoding

In [ ]:
# Define a function to conditionally one-hot encode a variable if it exists
def conditional_one_hot_encode(var_name, num_classes):
    if var_name in globals() and globals()[var_name] is not None:
        globals()[var_name] = tfk.utils.to_categorical(globals()[var_name], num_classes=num_classes)


# Conditionally apply one-hot encoding to each variable
conditional_one_hot_encode('y_test_aug', num_classes)
conditional_one_hot_encode('y_test_aug2', num_classes)
conditional_one_hot_encode('y_test_aug3', num_classes)
conditional_one_hot_encode('y_test_aug4', num_classes)
conditional_one_hot_encode('y_test_concat', num_classes)
conditional_one_hot_encode('y_test_concat2', num_classes)
conditional_one_hot_encode('y_test_concat3', num_classes)
conditional_one_hot_encode('y_test_concat_d', num_classes)

### Define The First - Model

In [ ]:
# Initialise MobileNetV3Small model with pretrained weights, for transfer learning
mobilenet = tf.keras.applications.EfficientNetV2B3(
    include_top=False,
    input_shape=(96, 96, 3),
    weights="imagenet",
    input_tensor=None,
    pooling=False,
    classes=8,
    classifier_activation="softmax",
    name="efficientnetv2-b3",
)


# Display a summary of the model architecture
mobilenet.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
# Specify 'to_file' argument with a path where you have write permissions
tfk.utils.plot_model(mobilenet, to_file='/tmp/model.png', expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Freeze all layers in MobileNetV3Small to use it solely as a feature extractor
mobilenet.trainable = False

# Define input layer with shape matching the input images
inputs = tfk.Input(shape=(96, 96, 3), name='input_layer')

"""
# Definisci il pipeline completo di augmentazione
augmentation = tf.keras.Sequential([
    # Altre augmentazioni indipendenti
    tfkl.RandomCrop(height=96, width=96),  # Regola la dimensione del crop se necessario
    tfkl.RandomFlip("horizontal_and_vertical"),
    tfkl.RandomRotation(0.3),
    tfkl.Dropout(0.1),
    tfkl.Dropout(0.2),
    tfkl.RandomContrast(0.3),
    tfkl.RandomZoom(0.15),
    tfkl.RandomBrightness(0.1),
], name='advanced_preprocessing')


#Apply the augmentation pipeline
inputs = augmentation(inputs)
"""

# Pass augmented inputs through the MobileNetV3Small feature extractor
x = mobilenet(inputs)

x = tfkl.GlobalAveragePooling2D(name='avg_pool')(x)

# Add a batch normalization layer
x = tfkl.BatchNormalization(name='batch_norm')(x)

# Add a dropout layer for regularization
x = tfkl.Dropout(0.1, name='dropout')(x)

# Add a dense layer with 256 units and GELU activation
x = tfkl.Dense(256, activation='gelu', name='dense1')(x)


# Add layer normalizatiFinal_Project.ipynbon
x = tfkl.LayerNormalization(name='layer_norm1')(x)

# Add another dropout layer
x = tfkl.Dropout(0.1, name='dropout2')(x)

# Add a second dense layer with 128 units and GELU activation
x = tfkl.Dense(128, activation='gelu', name='dense2')(x)

# Add layer normalization
x = tfkl.LayerNormalization(name='layer_norm2')(x)

# Add another dropout layer
x = tfkl.Dropout(0.1, name='dropout3')(x)

# Add a third dense layer with 128 units and GELU activation
x = tfkl.Dense(128, activation='gelu', name='dense3')(x)
'''
# Add layer normalization
x = tfkl.LayerNormalization(name='layer_norm3')(x)

# Add another dropout layer
x = tfkl.Dropout(0.3, name='dropout4')(x)
'''
# Add final Dense layer for classification with softmax activation
outputs = tfkl.Dense(8, activation='softmax', name='output')(x)

# Define the complete model linking input and output
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model with categorical cross-entropy loss and Lion optimiser
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Lion(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.99,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name="lion"
), metrics=['accuracy'])

# Display a summary of the model architecture
tl_model.summary(expand_nested=True)

### Train First - Model


In [ ]:
# Train the model
tl_history = tl_model.fit(
    x=X_test_concat3,
    y=y_test_concat3,
    batch_size=64,
    epochs=7,
    validation_data=(X_test_aug2 , y_test_aug2),
    callbacks=[tfk.callbacks.EarlyStopping(
        monitor='val_accuracy', 
        mode='max', patience=20,
        restore_best_weights=True
        )]
).history

# Calculate and print the best validation accuracy achieved
final_val_accuracy = round(max(tl_history['val_accuracy']) * 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file, including final accuracy in the filename
model_filename = 'Blood_Cells_MobileNetV3S_' + str(final_val_accuracy) + '.keras'
tl_model.save(model_filename)

### Test the First - Model


In [ ]:
# Generate predictions on the test set and print a classification report
y_pred = tl_model.predict(X_test_aug3)
y_pred_classes = y_pred.argmax(axis=1)  # Convert probabilities to class labels
y_test_classes = y_test_aug3.argmax(axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

del tl_model

### First Fine - Tuning


In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_76.54.keras')
#ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_'+ str(final_val_accuracy) + '.keras')

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Set the MobileNetV3Small model layers as trainable
ft_model.get_layer('efficientnetv2-b3').trainable = True

# Set all MobileNetV3Small layers as non-trainable
for layer in ft_model.get_layer('efficientnetv2-b3').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

In [ ]:
# Set the number of layers to freeze
N = 100

# Set the first N layers as non-trainable
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
    print(i, layer.name, layer.trainable)

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Lion(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.99,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name="lion"
), metrics=['accuracy'])

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = X_test_concat_d,
    y = y_test_concat_d,
    batch_size = 32,
    epochs = 6,
    validation_data = (X_test_aug2, y_test_aug2),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(ft_history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'Blood_Cells_MobileNetV3S_'+str(final_val_accuracy)+'.keras'
ft_model.save(model_filename)

del ft_model

### Second Fine - Tuning

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_76.54.keras')
#ft_model = tfk.models.load_model('Blood_Cells_MobileNetV3S_'+ str(final_val_accuracy) + '.keras')

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
#tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
# Set the MobileNetV3Small model layers as trainable
ft_model.get_layer('efficientnetv2-b3').trainable = True

# Set all MobileNetV3Small layers as non-trainable
for layer in ft_model.get_layer('efficientnetv2-b3').layers:
    layer.trainable = False

# Enable training only for Conv2D and DepthwiseConv2D layers
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.DepthwiseConv2D):
        layer.trainable = True
        print(i, layer.name, type(layer).__name__, layer.trainable)

In [ ]:
# Set the number of layers to freeze
N = 124

# Set the first N layers as non-trainable
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers[:N]):
    layer.trainable = False

# Print layer indices, names, and trainability status
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-b3').layers):
    print(i, layer.name, layer.trainable)

# Display a summary of the model architecture
ft_model.summary(expand_nested=True)

# Display model architecture with layer shapes and trainable parameters
tfk.utils.plot_model(ft_model, expand_nested=True, show_trainable=True, show_shapes=True, dpi=70)

In [ ]:
import tensorflow as tf

class Ranger(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=1e-5, weight_decay=1e-4, sync_period=5, slow_step=0.5, name="Ranger", **kwargs):
        super(Ranger, self).__init__(name=name, learning_rate=learning_rate, **kwargs)
        
        # Set other parameters
        self.weight_decay = weight_decay
        self.sync_period = sync_period
        self.slow_step = slow_step
        self.iterations = tf.Variable(0, dtype=tf.int64, trainable=False)
        
        # Lookahead slow weights initialization
        self.slow_weights = None

    def apply_gradients(self, grads_and_vars, name=None, **kwargs):
        # Get the learning rate directly from hyperparameters
        learning_rate = self._get_hyper("learning_rate")
        
        # Apply gradients with weight decay and learning rate
        for grad, var in grads_and_vars:
            if grad is not None:
                if self.weight_decay > 0:
                    grad += self.weight_decay * var
                var.assign_sub(learning_rate * grad)

        # Initialize slow weights on the first call
        if self.slow_weights is None:
            self.slow_weights = [tf.Variable(v, trainable=False) for _, v in grads_and_vars]

        # Increment the iteration counter
        self.iterations.assign_add(1)

        # Apply Lookahead every sync_period steps
        if tf.equal(self.iterations % self.sync_period, 0):
            for slow_var, (_, var) in zip(self.slow_weights, grads_and_vars):
                slow_var.assign_add(self.slow_step * (var - slow_var))
                var.assign(slow_var)

    def get_config(self):
        config = super(Ranger, self).get_config()
        config.update({
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "weight_decay": self.weight_decay,
            "sync_period": self.sync_period,
            "slow_step": self.slow_step
        })
        return config

# Instantiate and use the custom Ranger optimizer
optimizer = Ranger(learning_rate=1e-5, weight_decay=1e-4, sync_period=5, slow_step=0.5)

# Compile the model with the custom Ranger optimizer
ft_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = X_test_concat2,
    y = y_test_concat2,
    batch_size = 32,
    epochs = 5,
    validation_data = (X_test_aug4, y_test_aug4),
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=20, restore_best_weights=True)]
).history

# Calculate and print the final validation accuracy
final_val_accuracy = round(max(ft_history['val_accuracy'])* 100, 2)
print(f'Final validation accuracy: {final_val_accuracy}%')

# Save the trained model to a file with the accuracy included in the filename
model_filename = 'Blood_Cells_MobileNetV3S_'+str(final_val_accuracy)+'.keras'
ft_model.save(model_filename)

del ft_model

### Submit Section


In [ ]:
# file: model.py
class Model:
    def __init__(self):
        """Initialize the internal state of the model."""

    def predict(self, X):
        """Return a numpy array with the labels corresponding to the input X."""

In [ ]:
%%writefile model.py
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
import cv2

class Model:
    def __init__(self):
        """
        Initialize the internal state of the model.
        """
        self.neural_network = tfk.models.load_model('Blood_Cells_MobileNetV3S_77.45.keras')

    def kmeans_segmentation(self, images, k=4):
        """
        Apply k-means segmentation to a dataset of images.

        Parameters:
        - images: Numpy array of shape (n_samples, 96, 96, 3).
        - k: Number of clusters for k-means.

        Returns:
        - segmented_images: Images segmented by k-means (k=4) specified for given dataset.
        """
        segmented_images = []

        for image in images:
            # reshape into 2d np array for rgb
            pixel_values = image.reshape((-1, 3))
            pixel_values = np.float32(pixel_values)

            criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
            _, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

            # map pixels to cluster centers
            centers = np.uint8(centers)
            segmented_image = centers[labels.flatten()]
            segmented_image = segmented_image.reshape(image.shape)

            segmented_images.append(segmented_image)

        return np.array(segmented_images)

    def predict(self, X):
        """
        Predict the labels corresponding to the input X.

        Parameters:
        - X: Input images of shape (n_samples, 96, 96, 3).

        Returns:
        - preds: Predicted labels of shape (n_samples,).
        """
        # k-means segmentation
        kmeans_X = self.kmeans_segmentation(X, k=4)

        # Predict using the neural network
        preds = self.neural_network.predict(kmeans_X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds


In [ ]:
from datetime import datetime
filename = f'Outputs/submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
# The original path was incorrect. Using f-string to format correctly.
!zip {filename} model.py Blood_Cells_MobileNetV3S_77.45.keras